In [15]:
from tensorflow import keras

In [52]:
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
#from tensorflow.examples.tutorials.mnist import input_data
tf.compat.v1.enable_eager_execution()

ValueError: tf.enable_eager_execution must be called at program startup.

In [64]:
(x_train, y_train), (x_test, y_test) = tfds.load('fashion_mnist')

ValueError: too many values to unpack (expected 2)

In [65]:
(x_train, y_train), (x_test, y_test) =tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [67]:
X_train2 = tf.keras.datasets.fashion_mnist.load_data()

In [68]:
X_train2

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [18]:
#Training parameters
lr = 0.0002
batch_size = 128
epochs = 10000

#Network Parameters
img_dim = 784
gen_hid_dim = 256
disc_hid_dim = 256
z_noise_dim = 100

In [19]:
def xavier_init(shape):
    return tf.random.normal(shape, 1/tf.sqrt(shape[0]/2))

In [20]:
#weight and biases
weights = {'disc_H': tf.Variable(xavier_init([img_dim, disc_hid_dim]), dtype=tf.float32),
         'disc_final': tf.Variable(xavier_init([disc_hid_dim, 1]), dtype=tf.float32),
         'gen_H': tf.Variable(xavier_init([z_noise_dim, gen_hid_dim]), dtype=tf.float32),
          'gen_final': tf.Variable(xavier_init([gen_hid_dim, img_dim]), dtype=tf.float32)
         }

bias = {
        'disc_H': tf.Variable(xavier_init([disc_hid_dim]), dtype=tf.float32),
         'disc_final': tf.Variable(xavier_init([1]), dtype=tf.float32),
         'gen_H': tf.Variable(xavier_init([gen_hid_dim]), dtype=tf.float32),
          'gen_final': tf.Variable(xavier_init([img_dim]), dtype=tf.float32)
    }
            

In [21]:
#discriminator
def disc(x):
    hid_layer = tf.nn.relu(tf.add(tf.matmul(x, weights['disc_H']), bias['disc_H']))
    final_layer = tf.add(tf.matmul(hid_layer, weights['disc_final']), bias['disc_final'])
    disc_output = tf.nn.sigmoid(final_layer)
    return final_layer, disc_output
    

In [22]:
#generator
def gen(x):
    hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, weights['gen_H']), bias['gen_H']))
    final_layer = tf.add(tf.matmul(hidden_layer, weights['gen_final']), bias['gen_final'])
    gen_output = tf.sigmoid(final_layer)
    return gen_output

In [23]:
#tf.compat.v1.disable_eager_execution()

In [24]:
#place holder for external input
z_input = tf.compat.v1.placeholder(tf.float32, shape= [None, z_noise_dim], name = 'input_noise')
x_input = tf.compat.v1.placeholder(tf.float32, shape= [None, img_dim], name= 'real_input')

In [41]:
#buld generator network
with tf.name_scope('Generator') as scope:
    output_Gen = gen(z_input)

In [42]:
#buildin of discriminator
with tf.name_scope('Discriminator') as scope:
    real_output_disc, real_output_disc = disc(x_input)
    fake_output_disc, fake_output_disc = disc(output_Gen)

In [43]:
with tf.name_scope('Discriminator_loss') as scope:
    Discriminator_loss = -tf.reduce_mean(tf.log(real_output_disc + 0.0001) + tf.log(1 -fake_output_disc+0.0001))

In [44]:
with tf.name_scope('Generator_loss'):
    Generator_loss = -tf.reduce_mean(tf.log(fake_output_disc+0.0001))

In [45]:
disc_loss_total = tf.summary.scalar('Disc_Total_Loss', Discriminator_loss)
gen_loss_total = tf.summary.scalar('Gen_Loss', Generator_loss)

In [46]:
#variables
Generator_var = [weights['gen_H'], weights['gen_final'], bias['gen_H'], bias['gen_final']]
Discriminator_var = [weights['disc_H'], weights['disc_final'], bias['disc_H'], bias['disc_final']]

In [47]:
#dfn optimizer
with tf.name_scope('Optimizer_Discriminator') as scope:
    Discriminator_optimize = tf.train.AdamOptimizer(learning_rate = lr).minimize(Discriminator_loss, var_list = Discriminator_var)

In [48]:
#dfn optimizer
with tf.name_scope('Optimizer_Discriminator') as scope:
    Generator_optimize = tf.train.AdamOptimizer(learning_rate = lr).minimize(Discriminator_loss, var_list = Discriminator_var)

In [49]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
writer = tf.summary.FileWriter('./log', sess.graph)

In [66]:
for epoch in range(epochs):
    x_batch, _=tf.train.batch([x_train, y_train], batch_size=batch_size)
    
    #generate noise to feed to the neural network
    z_noise = np.random.uniform(-1, 1, size=[batch_size, z_noise_dim])
    _, Disc_loss_epoch = sess.run([Discriminator_optimize, Discriminator_loss], feed_dict={x_input:x_batch, z_input:z_noise})
    _, Gen_loss_epoch = sess.run([Generator_optimize, Generator_loss], feed_dict={z_input:z_noise})
    
    #running discriminator summary
    summary_Disc_Loss = sess.run(disc_loss_total, feed_dict={x_input:x_batch, z_input:z_noise})
    #adding discriminator summary
    writer.add_summary(summary_Disc_Loss, epoch)
    
    #run generator summary
    summary_Gen_Loss = sess.run(gen_loss_total, feed_dict={z_input:z_noise})
    #adding generator summary
    writer.add_summary(summary_Gen_Loss)
    
    if epoch% 2000 ==0:
        print(f'steps :{epoch} : Generator Loss : {Gen_loss_epoch}, Discriminator : {Disc_loss_epoch}')

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles. For reference, the tensor object was Tensor("batch:0", shape=(128, 60000, 28, 28), dtype=uint8) which was passed to the argument `feed_dict` with key Tensor("real_input_1:0", shape=(None, 784), dtype=float32).

In [ ]:
n = 6
canvas = np.empty((28*n, 28*n))
for i in range(n):
    #noise input
    z_noise = np.random.uniform(-1, 1, size= [batch_size, z_noise_dim])
    #generator image from noise
    g = sess.run(output_Gen, feed_dict{z_input:z_noise})
    #reverse colors for better display
    g = -1*(g-1)
    for j in range(n):
        #draw the generated digits
        canvas[i*28:(i+1)*28, j*28: (j+1)*28] = g[j].reshape([28, 28])

plt.figure(figsize=(n, n))
plt.imshow(canvas, origin='upper', cmap='gray')
plt.show()

In [61]:
f_mnist

({'train': <PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>,
  'test': <PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>},
 tfds.core.DatasetInfo(
     name='fashion_mnist',
     full_name='fashion_mnist/3.0.1',
     description="""
     Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
     """,
     homepage='https://github.com/zalandoresearch/fashion-mnist',
     data_path='C:\\Users\\morara\\tensorflow_datasets\\fashion_mnist\\3.0.1',
     file_format=tfrecord,
     download_size=29.45 MiB,
     dataset_size=36.42 MiB,
     features=FeaturesDict({
         'image': Image(shape=(28, 28, 1), dtype=uint8),


In [70]:
data = [i for i in range(40)]
data = np.array(data)

In [118]:
data = data.reshape(-1, 4, 10, 1)
data

array([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]]])